In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

# 1. Setting PySpark up

In [3]:
spark = SparkSession\
            .builder\
            .appName("HW5")\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 23:35:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 2. Load data

(Пришлось пошаманить, потому что спарк не хотел нормально открывать .csv с тройными кавычками, pandas такое легко открывает)

In [4]:
fp = "./train.csv"

schema = StructType(
    [
        StructField("id", StringType()),
        StructField("comment_text", StringType()),
        StructField("toxic", ByteType()),
        StructField("severe_toxic", ByteType()),
        StructField("obscene", ByteType()),
        StructField("threat", ByteType()),
        StructField("insult", ByteType()),
        StructField("identity_hate", ByteType())
    ]
)

df_train = spark.read.csv(fp, sep=",", header=True, quote="\"", escape="\"",
                         schema=schema)

In [5]:
df_train.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: byte (nullable = true)
 |-- severe_toxic: byte (nullable = true)
 |-- obscene: byte (nullable = true)
 |-- threat: byte (nullable = true)
 |-- insult: byte (nullable = true)
 |-- identity_hate: byte (nullable = true)



In [6]:
df_train.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|More I can't make...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
|00025465d4725e87|Congratulations f...|    0|           0|      0|     0|     0|            0|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|
|00031b1e95af7921|Your vandalism to...|    0|     

In [7]:
df_train.count()

159571

# 3. TF-IDF

Подготовим следующий пайплайн:
- Tokenizer
- StopWordsRemover
- HashingTF
- IDF

In [8]:
df_train = df_train.withColumn("comment_text", F.lower(F.col("comment_text")))\
                    .na.drop()

In [9]:
df_train.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: byte (nullable = true)
 |-- severe_toxic: byte (nullable = true)
 |-- obscene: byte (nullable = true)
 |-- threat: byte (nullable = true)
 |-- insult: byte (nullable = true)
 |-- identity_hate: byte (nullable = true)



In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.pipeline import Pipeline

In [11]:
def create_pipe(labelCol="toxic"):
    tokenizer = Tokenizer(inputCol="comment_text", outputCol="tokenized")
    stopwords = StopWordsRemover.loadDefaultStopWords("english")
    remover = StopWordsRemover(stopWords=stopwords).setInputCol("tokenized").setOutputCol("tok_rem")
    hashing_tf = HashingTF(inputCol="tok_rem", outputCol="hashed")
    idf = IDF(inputCol="hashed", outputCol="idf")
    lr = LogisticRegression(featuresCol="idf", labelCol=labelCol)
    pipe = Pipeline(stages=[tokenizer, remover, hashing_tf, idf, lr])
    grid = ParamGridBuilder().addGrid(lr.maxIter, [100])\
                            .addGrid(lr.regParam, [0.01, 0.1, 1.0])\
                            .addGrid(lr.elasticNetParam, [0, 0.1, 1.0])\
                            .addGrid(hashing_tf.numFeatures, [1000, 30000, 100000]).build()
    evaluator = BinaryClassificationEvaluator(labelCol=labelCol)
    cv = CrossValidator(estimator=pipe, estimatorParamMaps=grid, evaluator=evaluator, parallelism=4)
    cvModel = cv.fit(df_train)
    return evaluator, pipe, cvModel

## 3.1. Toxic

In [14]:
%%time

ev_tox, pipe_tox, cvModel_tox = create_pipe(labelCol="toxic")

22/12/06 20:46:48 WARN StopWordsRemover: Default locale set was [en_RU]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/12/06 20:46:48 WARN StopWordsRemover: Default locale set was [en_RU]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/12/06 20:46:48 WARN StopWordsRemover: Default locale set was [en_RU]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/12/06 20:46:48 WARN StopWordsRemover: Default locale set was [en_RU]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/12/06 20:46:48 WARN StopWordsRemover: Default locale set was [en_RU]; however, it was not found in available locales in JVM, falling back to en_U

22/12/06 20:46:57 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/12/06 20:46:58 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


22/12/06 20:47:01 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/12/06 20:47:02 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/06 20:47:02 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


...

22/12/06 20:56:38 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/12/06 20:56:41 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:41 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:44 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:45 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:45 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:45 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:45 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:46 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:56:46 WARN DAGScheduler: Broadcasting larg

22/12/06 20:57:01 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:01 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:02 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:03 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:03 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:03 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
22/12/06 20:57:03 WARN DAGScheduler: Broadcasting larg

Лучшая метрика (ROC-AUC):

In [31]:
max(cvModel_tox.avgMetrics)

0.9095645382544455

In [29]:
cvModel_tox.getEstimatorParamMaps()[ np.argmax(cvModel_tox.avgMetrics) ]

{Param(parent='LogisticRegression_098d47a4f621', name='maxIter', doc='max number of iterations (>= 0).'): 100,
 Param(parent='LogisticRegression_098d47a4f621', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LogisticRegression_098d47a4f621', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1,
 Param(parent='HashingTF_b785aa439b6e', name='numFeatures', doc='Number of features. Should be greater than 0.'): 250000}

# 3.2. Other targets

In [12]:
spark.sparkContext.setLogLevel('ERROR')

In [13]:
label_columns = ["severe_toxic", "obscene", "threat", "insult", "identity_hate"]
results = {}

for labelCol in label_columns:
    ev, pipe, cvModel = create_pipe(labelCol=labelCol)
    results[labelCol] = cvModel
    print(f"Качество {labelCol}: {max(cvModel.avgMetrics)}")
    print(f"Лучшие параметры:")
    print(cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ])
    print("----------")

Качество severe_toxic: 0.9302449048818756
Лучшие параметры:
{Param(parent='LogisticRegression_a8346dea75d5', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LogisticRegression_a8346dea75d5', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_a8346dea75d5', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1, Param(parent='HashingTF_1f033b2dfd7f', name='numFeatures', doc='Number of features. Should be greater than 0.'): 100000}
----------


Качество obscene: 0.9210393555069746
Лучшие параметры:
{Param(parent='LogisticRegression_34c559666e86', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LogisticRegression_34c559666e86', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_34c559666e86', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1, Param(parent='HashingTF_ba9337a2ee03', name='numFeatures', doc='Number of features. Should be greater than 0.'): 100000}
----------


Качество threat: 0.9255329092266793
Лучшие параметры:
{Param(parent='LogisticRegression_d82f26c927cd', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LogisticRegression_d82f26c927cd', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='LogisticRegression_d82f26c927cd', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='HashingTF_c54a8ce4898f', name='numFeatures', doc='Number of features. Should be greater than 0.'): 100000}
----------


Качество insult: 0.9222137760698069
Лучшие параметры:
{Param(parent='LogisticRegression_7882cbb530a8', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LogisticRegression_7882cbb530a8', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_7882cbb530a8', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1, Param(parent='HashingTF_3a8e25e77b30', name='numFeatures', doc='Number of features. Should be greater than 0.'): 100000}
----------


Качество identity_hate: 0.8957673607978274
Лучшие параметры:
{Param(parent='LogisticRegression_0c580653454a', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LogisticRegression_0c580653454a', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='LogisticRegression_0c580653454a', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='HashingTF_bafec11773d3', name='numFeatures', doc='Number of features. Should be greater than 0.'): 30000}
----------


**Выводы:** one-vs-all бинарная классификация оказалась достаточно легкой задачей, качество везде почти 0.9 AUC. В HashingTF чем больше numParameters, тем лучше (больше оставляем слов).

# 4. Word2Vec

In [13]:
def create_pipe_w2v(labelCol="toxic"):
    tokenizer = Tokenizer(inputCol="comment_text", outputCol="tokenized")
    stopwords = StopWordsRemover.loadDefaultStopWords("english")
    remover = StopWordsRemover(stopWords=stopwords).setInputCol("tokenized").setOutputCol("tok_rem")
    w2v = Word2Vec(inputCol="tok_rem", outputCol="w2v")
    lr = LogisticRegression(featuresCol="w2v", labelCol=labelCol, regParam=0.01, maxIter=100)
    pipe = Pipeline(stages=[tokenizer, remover, w2v, lr])
    grid = ParamGridBuilder().addGrid(w2v.windowSize, [3, 5, 7])\
                            .addGrid(lr.elasticNetParam, [0, 0.1])\
                            .addGrid(w2v.vectorSize, [50, 100])\
                            .build()
    evaluator = BinaryClassificationEvaluator(labelCol=labelCol)
    cv = CrossValidator(estimator=pipe, estimatorParamMaps=grid, evaluator=evaluator, parallelism=4)
    cvModel = cv.fit(df_train)
    return evaluator, pipe, cvModel

**IMPORTANT:** так как часто падало из-за memory heap overflow, то приходилось распечатывать результаты и перезапускать для оставшихся моделей.

In [14]:
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
results_w2v = {}

for labelCol in tqdm(label_columns[:]):
    ev, pipe, cvModel = create_pipe_w2v(labelCol=labelCol)
    results_w2v[labelCol] = cvModel
    print(f"Качество {labelCol}: {max(cvModel.avgMetrics)}")
    print(f"Лучшие параметры:")
    print(cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ])
    print("----------")

100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [13:51<00:00, 831.19s/it]

Качество identity_hate: 0.9445430153867115
Лучшие параметры:
{Param(parent='Word2Vec_4515953a1fd6', name='windowSize', doc='the window size (context words from [-window, window]). Default value is 5'): 7, Param(parent='LogisticRegression_e220752812bd', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1, Param(parent='Word2Vec_4515953a1fd6', name='vectorSize', doc='the dimension of codes after transforming from words'): 100}
----------


In [19]:
for labelCol, cvModel in results_w2v.items():
    cvModel = results_w2v[labelCol]
    print(f"Качество {labelCol}: {max(cvModel.avgMetrics)}")
    print(f"Лучшие параметры:")
    print(cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ])
    print("----------")

Качество severe_toxic: 0.9763800151854604
Лучшие параметры:
{Param(parent='Word2Vec_b54812016af2', name='windowSize', doc='the window size (context words from [-window, window]). Default value is 5'): 7, Param(parent='LogisticRegression_b7ef6b6c3ae0', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1, Param(parent='Word2Vec_b54812016af2', name='vectorSize', doc='the dimension of codes after transforming from words'): 100}
----------


In [15]:
for labelCol, cvModel in results_w2v.items():
    cvModel = results_w2v[labelCol]
    print(f"Качество {labelCol}: {max(cvModel.avgMetrics)}")
    print(f"Лучшие параметры:")
    print(cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ])
    print("----------")

Качество obscene: 0.9567709461331929
Лучшие параметры:
{Param(parent='Word2Vec_12e3b7ea6682', name='windowSize', doc='the window size (context words from [-window, window]). Default value is 5'): 7, Param(parent='LogisticRegression_ef6c6dd7061f', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='Word2Vec_12e3b7ea6682', name='vectorSize', doc='the dimension of codes after transforming from words'): 100}
----------
Качество threat: 0.9485366784161625
Лучшие параметры:
{Param(parent='Word2Vec_d380a2c66d4c', name='windowSize', doc='the window size (context words from [-window, window]). Default value is 5'): 7, Param(parent='LogisticRegression_090ce9813e78', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='Word2Vec_d380a2c66d4c',

**Вывод:** кажется, что результаты еще выросли ~3-5% в зависимости от случая. В целом оказалось, что окно лучше брать достаточно большое (5), размер вектора - побольше (с 300 пробовал, но тоже по памяти плохо было при CV).